In [ ]:
import datetime
import calendar
import json

# item list
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": 10,
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": 35000,
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": "7000",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
        "idx": 16,
        "item_code": "Breakout Area",
        "qty": 3,
        "rate": "3000",
        "amount": "9000",
        "start_date": "2024-01-01",
        "stop_date": "2024-01-31",
    }
]

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    # Parse target month
    year, month = map(int, target_month.split('-'))
    
    _, last_day = calendar.monthrange(year, month)
    month_start = datetime.date(year, month, 1)
    month_end = datetime.date(year, month, last_day)
    
    result = {
        "line_items": [],
        "total_revenue": 0.0
    }
    
    grouped_items = {}
    
    # Process each item
    for item in item_list:
        start_date = datetime.datetime.strptime(item["start_date"], "%Y-%m-%d").date()
        stop_date = datetime.datetime.strptime(item["stop_date"], "%Y-%m-%d").date()
        
        if start_date > month_end or stop_date < month_start:
            continue  
        
        billable_start = max(start_date, month_start)
        billable_end = min(stop_date, month_end)
        billing_period = f"{billable_start.strftime('%Y-%m-%d')} to {billable_end.strftime('%Y-%m-%d')}"
        
        rate = float(item["rate"])
        qty = int(item["qty"])
        
        group_key = (item["item_code"], rate, billing_period)
        
        if group_key in grouped_items:
            grouped_items[group_key]["qty"] += qty
        else:
            grouped_items[group_key] = {
                "item_code": item["item_code"],
                "rate": rate,
                "qty": qty,
                "billing_period": billing_period,
                "billable_start": billable_start,
                "billable_end": billable_end
            }
    
    for group_data in grouped_items.values():
        days_in_billing_period = (group_data["billable_end"] - group_data["billable_start"]).days + 1
        
        total_days_in_month = (month_end - month_start).days + 1
        
        full_month_amount = group_data["rate"] * group_data["qty"]
        amount = full_month_amount * (days_in_billing_period / total_days_in_month)
        
        line_item = {
            "item_code": group_data["item_code"],
            "rate": group_data["rate"],
            "qty": group_data["qty"],
            "amount": amount,
            "billing_period": group_data["billing_period"]
        }
        
        result["line_items"].append(line_item)
        result["total_revenue"] += amount
    
    for line_item in result["line_items"]:
        line_item["amount"] = round(line_item["amount"], 1)
    
    result["total_revenue"] = round(result["total_revenue"], 1)
    
    return result

def is_valid_month_format(month_str):
    """Validate if the input string matches YYYY-MM format."""
    try:
        # Check format
        if len(month_str) != 7 or month_str[4] != '-':
            return False
        
        year, month = map(int, month_str.split('-'))
        if not (1000 <= year <= 9999 and 1 <= month <= 12):
            return False
            
        return True
    except:
        return False

def main():
    print("Monthly Bill Generator")
    print("======================")
    
    while True:
        target_month = input("\nEnter month to generate bill for (YYYY-MM format, e.g., 2024-11) or 'exit' to quit: ")
        
        if target_month.lower() == 'exit':
            print("Exiting program. Goodbye!")
            break
            
        if not is_valid_month_format(target_month):
            print("Error: Invalid month format. Please use YYYY-MM format (e.g., 2024-11).")
            continue
            
        try:
            bill = generate_monthly_bill(item_list, target_month)
            
            year_month = datetime.datetime.strptime(target_month, "%Y-%m")
            month_name = year_month.strftime("%B %Y")
            
            print(f"\nMonthly Bill for {month_name}:")
            print("=" * (len(month_name) + 16))
            
            if not bill["line_items"]:
                print("No active items found for this month.")
            else:
                print(json.dumps(bill, indent=2))
                
        except Exception as e:
            print(f"Error generating bill: {e}")

if __name__ == "__main__":
    main()